<a href="https://colab.research.google.com/github/Roger8886/Taiwan-stock-prediction/blob/main/Taiwan_stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 安裝twstock模組
!pip install twstock

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 6.3 MB/s 


In [2]:
# 載入twstock模組
import twstock

In [3]:
# 在twstock模組中利用 Stock 方法查詢個股歷史股票資料
stock = twstock.Stock('2317')
print(stock.price)

[105.0, 104.0, 103.5, 105.5, 106.0, 107.5, 108.0, 108.0, 108.5, 109.0, 109.0, 108.0, 108.5, 108.0, 108.0, 108.0, 109.0, 110.0, 113.0, 112.5, 112.0, 111.0, 112.5, 112.0, 111.0, 111.0, 110.0, 109.5, 109.5, 110.5, 108.0]


In [4]:
# 顯示最近一日的開盤價、最高價、最低價、收盤價
print("日期:", stock.date[-1])
print("開盤價：", stock.open[-1])
print("最高價：", stock.high[-1])
print("最低價：", stock.low[-1])
print("收盤價：", stock.price[-1])

日期: 2022-08-29 00:00:00
開盤價： 108.5
最高價： 109.0
最低價： 108.0
收盤價： 108.0


In [5]:
# 利用stock物件中的fetch()方法讀取指定期間的歷史資料
stock.fetch(2022,7)

[Data(date=datetime.datetime(2022, 7, 1, 0, 0), capacity=63133602, turnover=6758749961, open=108.5, high=109.0, low=105.5, close=106.0, change=-3.0, transaction=32535),
 Data(date=datetime.datetime(2022, 7, 4, 0, 0), capacity=70995980, turnover=7140989017, open=100.5, high=102.0, low=99.5, close=100.0, change=0.0, transaction=32374),
 Data(date=datetime.datetime(2022, 7, 5, 0, 0), capacity=45713186, turnover=4669649468, open=102.0, high=103.0, low=101.0, close=102.5, change=2.5, transaction=20422),
 Data(date=datetime.datetime(2022, 7, 6, 0, 0), capacity=45875699, turnover=4653042441, open=103.0, high=103.5, low=100.5, close=100.5, change=-2.0, transaction=22660),
 Data(date=datetime.datetime(2022, 7, 7, 0, 0), capacity=41084778, turnover=4210411576, open=101.0, high=103.5, low=100.5, close=103.5, change=3.0, transaction=15382),
 Data(date=datetime.datetime(2022, 7, 8, 0, 0), capacity=35186757, turnover=3613971179, open=103.5, high=104.0, low=101.5, close=102.0, change=-1.5, transactio

In [ ]:
# 讀取2021年12月到今天的資料
stock.fetch_from(2021,12)

In [7]:
# 下載個股全年資料
import csv, os, time
import twstock

datayear = 2021
startmonth = 1
filepath = 'twstock' + str(datayear) + '.csv'
title = ["日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數"]

for i in range(startmonth, 13):
    outputfile = open(filepath, 'a', newline='', encoding='big5')
    outputwriter = csv.writer(outputfile)
    stock = twstock.Stock('2317')
    stocklist=stock.fetch(datayear,i)
    data=[]
    for stock in stocklist:
        strdate = stock.date.strftime("%Y-%m-%d")
        li = [strdate,stock.capacity,stock.turnover,
              stock.open,stock.high,stock.low,stock.close,
              stock.change,stock.transaction]
        data.append(li)
    if i==1:
        outputwriter.writerow(title)
    time.sleep(2)
    outputfile.close()

ConnectionError: ignored